<a href="https://colab.research.google.com/github/paulodowd/Webots_2021/blob/master/WLabsheets/L7_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 7: Mapping

The e-puck robot has 8 IR proximity sensors around the robot body.  In Labsheet 4 you will have collected sample data from your proximity sensors, and then linearised the response.  This means that, instead of receiving a raw sensor value (the simulated analog to digital conversion), you can approximate a distance reading (e.g., in mm).  

In this labsheet, we will work through a suggestion on how to utilise this distance reading to add detected obstructions into a map.  We will utilise some advantages of the simulator, such as a large amount of working memory that we would not otherwise expect to see on a robot operated by a microcontroller.  It should be noted however, that large memory operations are likely to slow down the run-time of your simulator.





<hr><br><br><br><br>

# Considerations for a Metric Map

The most straight-forward map we can produce is a `metric map`, a grip of some fixed resolution sufficient to represent the space of the task environment.  In the Webots world provided, we can inspect in the left hand pane the field `RectangleArean "rectangle arena"` and see that it has a size `floorSize 0.9 0.9`, which is 900mm by 900mm.  In this labsheet, we'll work with the idea of using a grid of 90x90 cells, providing a representation of 10x10mm per map grid cell.  

<p align="center">
<img src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_9090Map.png?raw=true">
</p>

In the above illustration, we suppose that the 90x90 grid has a direct (but scaled) mapping to the arena environment.  For example, the corners of the grid match the corners of the arena.  Similarly, we might assume that cell `0,0` is the bottom left of the grid, whilst cell `89,89` is the top right.

If we consider the above illustration, we can consider some initial problems our robot may face:
- If the robot was to start in an unknown location, it cannot know where to place detected obstacles of features into a direct mapping of the metric map to the arena world.
  - For example, if the robot starts in the position in the graphic on the left, we would assume odometry (x,y) is initialised to `0,0` at this position.
- If the robot utilises the map with an unknown starting position, the robot may navigate out of the area represented by the metric map.

We can consider several solutions for this metric map problem:
- The robot could begin using the map, and then when the limits of the arena are detected, the entire map representation is updated with the correct coordinate offset to all mapped features.  
- The robot could utilise a map representation that covers an area large than the task environment, although this is inefficient.
- Different methods of mapping can be considered, such a topological or graph-based map.

Even with the above considerations, we have an issue about whether the robot can autonmously detect the arena boundaries, and issues of localisation such as it's rotation with respect to a given global coordinate system.  

To investigate in this area, it is sufficient to make some assumptions to gain an operational implementation, and then to challenge yourself to investigate the more challenging underlying problems within the assumptions.

<hr><br><br><br><br>

# Projecting a Point

From the point of view of the robot, when an obstruction is detected the robot can know:
- which sensor has detected the obstruction (sensor $s$).
- where this sensor is positioned on it's body (sensor angle, $\theta_{s}$, and distance $d_{s}$ from centre of robot point $P_{R}$).
- what the distance reading was (distance measurement, $M_{s}$).
- an estimate of the robot's current position, via odometry ($X_{I}$, $Y_{I}$ and $\theta_{I}$).

These four elements can be combined to allow the robot to add information into a map.  To add information into the map, relative to the robot's current position.  This can be clearly broken down into the following steps using simple trigonometry:

<p align="center">
<img width="50%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_SensorProjection1.png?raw=true">
</p>

1. Get a distance reading from the sensor, $M_{s}$ (measurement for sensor $s$).  We know that the sensor is located around the robot body by angle $\theta_{s}$, and offset from point $P_{R}$ by distance $d_{s}$.


<p align="center">
<br><br><br><br>
<img width="50%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_SensorProjection_Assumption.png?raw=true">
</p>

2. With an assumption that the robot centre point $P_{R}$ is located at $X_{I} = 0$, $Y_{I} = 0$, and including the rotational offset $\theta_{I}$, we can work out where point $P_{B}$ (here, B for block-obstruction) is located:

$$x = ( d_s + M_s ) * cos( \theta_{I} + \theta_{s} )$$
$$y = ( d_s + M_s ) * sin( \theta_{I} + \theta_{s} )$$



<p align="center">
<br><br><br><br>
<img width="50%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_SensorProjection2.png?raw=true">
</p>

3. Because of our initial assumption that $P_{R}$ is at $X_{I} = 0$, $Y_{I} = 0$, we must include the translation of $X_{I}$ and $Y_{I}$, to produce the final form of our equations:


$$ P_{B}(x) = X_I + (( d_s + M_s) * cos( \theta_{I} + \theta_{s}) )$$
$$ P_{B}(y) = Y_I + (( d_s + M_s) * sin( \theta_{I} + \theta_{s}) )$$



## Exercise 1: Testing a transformation into X,Y coordinates.

1. Building on your previous work, write the above equations into your Webots code to transform a sensor reading into a set of co-ordinates for the obstruction detected.  To being, just use `printf()` to validate the co-ordinates being output.
  - **Help:** it is necessary that you have previously linearised your proximity sensor readings into an estimate of distance.
  - **Help:** To start with, implement code to just transform 1 sensor reading and report the results.
  - **Validate:** Working with just 1 sensor, validate the transformation of sensor reading into `(x,y)` coordinates appears to be functioning properly.
  - **Help:** Remember that you can organise the environment so that you can evaluate your code against a known truth (e.g., place an obstacle a known distance away from your robot, set at 0,0).
  - **Help:** Use the documentation to find the distance of the IR Proximity sensors from the centre of the e-puck robot.
  - **Help:** The e-puck has the following sensor orientation in radians, although you should validate if these are in the correct order around the robot body:

```c   
float ps_dirs[8] = { 5.986479, 5.410521, 4.712389, 3.665191, 2.617994, 1.570796, 0.8726646, 0.296706};
```

<br><br>
2.  Once you have evaluated that you are able to transform 1 sensor reading into a set of `(x,y)` write two functions:

```c

// Returns X coordinate for a given sensor angle and measurment.
// Note, accessing the global odometry struct for global coordinates
float sensorReadingToX( float sensor_angle, float measurement ) {

  return x;
}

// Returns Y coordinate for a given sensor angle and measurment.
// Note, accessing the global odometry struct for global coordinates
float sensorReadingToY( float sensor_angle, float measurement ) {

  return y;
}
```


<br><br>
3. Test your two functions above by reading all 8 sensors within a `for loop`, and print the co-ordinates of obstructions using `printf()`:

```c
int sensor;
float x;
float y;

// For each sensor
for( sensor = 0; sensor < NB_PS; sensor++ ) {

    // Get x and y based on reading and sensor
    // angle.
    x = sensorReadingToX( ... );
    y = sensorReadingToY( ... );

    // print result.
    printf("Sensor %d = (%.2f, %.2f)\n", sensor, x, y );

}
```

<hr><br><br><br><br>

# Logging into a (map) Array in C



Within the scope of these labsheets, it will be sufficient for you to create a 2D array in `global scope` to store your map data.  We will then write some functions to encapsulate the operations we wish to perform on the map data.  

As previously stated, we will create a map of 90x90 cells.  We can achieve this with a the following line of code in the global scope of your Webots controller:

```c
// Define a 2D array to store our map data
#define MAP_XLIM 90
#define MAP_YLIM 90
float map[ MAP_XLIM ][ MAP_YLIM ];
```

In the above, we use `#define MAP_XLIM 90` and `#define MAP_YLIM 90` to set the limits of our array.  This will be useful later if we need to iterate over the array.  For example, we will be able to use code such as:

```c
int x;
int y;

for( y = 0; y < MAP_YLIM; y++ ) {    // start at row 0
  for( x = 0; x < MAP_XLIM; x++ ) {  // for all columns

      // Sets location x,y in array to 0.0
      map[x][y] = 0.0;

      // Reads and prints value at location x,y
      // to 2 decimal places.
      printf("%.2f, ", map[x][y] );

  }
  
  // line return here, to print out the 2d array
  // row by row
  printf("\n");

}
```

In the above code extract, we can be assured that the `for loop` will iterate from 0 to the limit of our map size, based on the value of `MAP_XLIM` and `MAP_YLIM` set at the top of our program.  This will reduce the likelihood of bugs or errors if these values were defined variously throughout your source code.  We can then also easily alter the size of our map by adjusting just `MAP_XLIM` and `MAP_YLIM`.

Once we have taken a sensor reading and converted it to `(x,y)` coordinates, we will want to add this to our map.  From the above code suggestion, your co-ordiantes will be floating point numbers, and these are not appropriate for indexing an array. Therefore, we will use `typecasting` to change the floating point value into an integer representation. If we want to be safe about the behaviour, we can first of all use the function `floor()` to round-down the floating point number:

```c
// To temporarily store x,y of obstruction.
float obs_x;
float obs_y;

// To temporarily store x,y index of map
int map_x;
int map_y;

// Use current sensor reading to get x,y
obs_x = sensorReadingToX( ... );
obs_y = sensorReadingToY( ... );

// We know that we are using a map 90x90
// to represent arena of 900x900.  Therefore,
// here we scale down, divide 10
// In your code, find a way to do this relating
// the global parameters MAP_XLIM MAP_YLIM
obs_x = obs_x / 10.0;
obs_y = obs_y / 10.0;

// Round down result
obs_x = floor( obs_x );
obs_y = floor( obs_y );

// Convert to int
map_x = (int)obs_x;
map_y = (int)obs_y;

// Check that map_x and map_y are within the
// limits of the map size. Decide what to do
// if not:
//
// if( ... ) {
//    ...
// }

// Increment this map location by 1.
map[map_x][map_y] = map[map_x][map_y] + 1;


```

In the above example code, we simply increment the value of the cell at location `map_x` and `map_y`.  This is called an `occupancy histogram`.  This type of map will record the sum total that a cell was registered as having an obstruction.  Cells with a zero value will be considered as unoccupied space.  






## Exercise 2: Mapping Functions

1. Write a function to initialise your map to a known value, such as all cells equal `0.0`.  Call this map initialising function from within your `setup()` function.

2. Write a useful function to call that will print all of the map to the console in a 2D format.  
  - Decide whether it prints from row 0 to MAP_YLIM, or from MAP_YLIM to 0.  Make a similar decision about the x or column formating.

3. **Validate**: validate that your map initialising function works as you expect.

4. Write a function to take an `x` and `y` value and record a new data entry into your map.  
  - It is suggested that you start by incrementing an existing value in the map array.  

5. **Validate:** check that your mapping system is working correctly so far.  
  - **Help:** You can use Webots to locate your e-puck robot at coordinates `0,0` in the arena, but set your odometry to start at `450, 450`.  This will centre your 2D map array over the arena.

6. **Validate:** check what happens to your program and your map when your robot reaches the limits of the arena, or exceeds the map array.  
  - **Help:** Try to write your code to catch these error exception by printing an error message, rather than allowing your program to crash.





<hr><br><br><br><br>

# Writing a Map out to a text file (.csv)

In this section, we will quickly review how to write the contents of your map array out to a text file stored on your computer disk.  We will actually use a text file with a `.csv` extension, which stands for `comma separated values`.  A `.csv` file is useful because it can be read into a table automatically by Microsoft Excel, R, Matlab, or other data processing software.

In the approach provided here, your file `my_map.csv` will keep a record of the map each time `writeMap()` is called.  For example, you will be able to review how the map changed after every time `writeMap()` is called.  For this reason, it is important that you do not call `writeMap()` on every update of the simulator, because this will quickly create a very large file on your computer.  It is therefore recommended to call `writeMap()` after some time interval, such as every second.

```c
// We should include stdlib.h
#include <stdlib.h>


// A file pointer, this will point to the location
// on your computer disk, where to write the data to.
FILE * fp;
char filename = "my_map.csv";


// We will open the text file in setup() with a special
// flag to delete any previous content.
// Setup will only run once, so we use this to renew
// our recorded map data.
void setup() {
  
  // Using "w" as an argument will overwrite an old
  // file with new content.
  fp = fopen( filename, "w" );

  // If we couldn't open the file, report an error.
  // This could be if you don't have permission to
  // use the current working directory, or if the
  // .csv file is held open by another program.
  if( fp == null ) {
      printf("Error opening %s\n", filename );
      
      // You should handle this error appropriately.
      // ...
  }

  // We now close the file, after creating it on
  // disk.  When we log our map data to the file,
  // we will open and close it every time.  We do
  // this because we do not know when the controller
  // program will end.  Therefore, we "save" the
  // contents of the file after every write operation
  // by closing it.
  if( fp != NULL ) fclose( fp );

}

void loop() {

  // You controller code.


  // We write the map out to our csv file.
  // Note, if you write out a map on every update
  // your file size will be very large!
  
  // if( time_interval has elapsed ) {
    writeMap();
  // }

}

void writeMap() {
  int x;
  int y;

  // Open the file to append (add new data)
  fp = fopen( filename, "a" );

  // If we couldn't open the file, report an error.
  // This could be if you don't have permission to
  // use the current working directory.
  if( fp == NULL ) {
      printf("Error opening %s\n", filename );
      return;
      
  }

  // Loop through map and print to file.
  for( y = 0; y < MAP_YLIM; y++ ) {
    for( x = 0; x < MAP_XLIM; x++ ) {

      // Similar to printf, except we direct
      // output to the the file pointer fp
      fprintf( fp, "%.2f, ", map[x][y] );
    }

    // Newline so we print a 2d array
    fprintf( fp, "\n" );
  }

  // Newline so that there is a gap between
  // map records in the csv file
  fprintf( fp, "\n" );

  // Close the file, saving the changes.
  if( fp != NULL ) fclose( fp );


}

```

## Exercise 3: Writing out Results

1.  Integrate the above example code into your Webots controller.  
  - **Validate:** Check that you can `restart` your controller program, and see a new `my_map.csv` file created on disk.  You might need to look at the number of bytes stored on disk to see the change.
  - **Validate:** Check that when you run your controller program, and then stop it, that you can see multiple versions of your map within `my_map.csv`.

2. Decide how to utilise this functionality:
  - Decide how frequently you should call `writeMap()`.  This will depend on the behaviour of your robot and the duration of your experiments.
  - If you use a resolution higher than `90x90`, your file size will increase proportionally.  
  - You may decide to instead only write out a map at the end of your controller program.  The simulator has no predefined end.  Thefeore, you may need to write some code that uses time or an objective to decide when your robot has "finished" it's task, and then write the map to disk.
  - You can use a similar method to write other information to disk for later analysis.  For example, you might want to create another `.csv` file to store information on odometry over time.
  - You can use the code in `setup()` to add initial headers to your `.csv` file where it is appropriate. Within `.csv`, column headers are usually encapsulated in double-quotation (`"`).  In C, you must use an escape character to create this formating:

```c
// Example of writing string headers to the .csv file pointer.
// backstroke allows " to be printed: e.g.  \"
fprintf( fp, " \"X result\", \"Y result\", \"Theta result\" ");

```

3. Allow your robot to navigate the environment and update it's map.  Validate that you are logging into the map appropriately.
  - **Help:** Remember that, for a resolution of 90x90 cells (therefore, 1 cell is 10mm), your robot must move a 10mm before data is entered into a new cell.  If your robot reports data into a cell on every call of the simulation, 1 cell will accumulate a lot of readings before the robot has moved to a new cell.

<hr><br><br><br><br>



# True-Positive, False-Positive, True-Negative, False-Negative

<p align="center">
<img width="75%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_MappingReadings.png?raw=true">
</p>

We know from a previous labsheet that when the e-puck does not have an obstruction within it's sensor view, the sensor will still report a range measurement.  This means that an immediate challenge to use the IR proximity sensors is to decide when to ignore sensor readings in the context of mapping.  

In the illustration above, a pink zone has been indicated where the robot cannot disciriminate between a reading without an obstruction, and when an obstruction is present.  If this region was logged into the map, the map would have a large number of `false-positive` readings - entries where an obstruction was recognised that did not exist in reality.

The green area of the sensors will be readings that we can have a higher degree of confidence of being true.  Unfortunately, the sensors have noise, so there is still a chance that we might add an obstruction to the map in a location that it does not exist (a `false-positive`).  However, in the green area, we are much more likely to log a `true-postive`, an obstruction where one exists in reality.

Because of sensor noise, it is possible that the robot may not detect an obstruction when one does exist in reality.  This is known as a `false-negative` - the robot has indicated free space (no obstruction detected) incorrectly.

When the robot does not detect an obstruction, and this is the correct state of reality, this is referred to as a `true-negative`.  







# Exercise 4: Discrimination

1.  Use techniques you have developed before to decide by threshold (here, of distance measured) whether a reading from a sensor should be logged into your map.  

2. Investigate a smaller arena size, allowing you to produce a higher resolution map.  
  - **Help:** you could use the drawing on the arena floor and line following to keep your epuck robot within a smaller area of the arena.

3. Investigate comparing your produced map against a known configuration of the environment (the `gound-truth`).  For example, create a smaller arena with an obstacle in a known location.  Compare your produced map against the ground-truth for the quantity of:
 - True-Positives
 - False-Positives
 - True-Negatives
 - False-Negatives

- If you are working in Excel, this can be as simple as subtracting cell data of one sheet from another.
- You may wish to investigate <a href="https://en.wikipedia.org/wiki/Sensitivity_and_specificity">other ways</a> of processing data into similar metrics.
